# UCI-HAR CNN

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical

import wandb
from wandb.keras import WandbMetricsLogger, WandbCallback

## Load and format UCI-HAR dataset

In [2]:
x_train = np.loadtxt('x_train.csv', delimiter=',').reshape((-1, 128, 9))
y_train = np.loadtxt('y_train.csv', delimiter=',')
x_test = np.loadtxt('x_test.csv', delimiter=',').reshape((-1, 128, 9))
y_test = np.loadtxt('y_test.csv', delimiter=',')
np.savetxt('x_test_250.csv', x_test.reshape((x_test.shape[0], -1))[0:250], delimiter=',', fmt='%s')
np.savetxt('y_test_250.csv', y_test[0:250], delimiter=',', fmt='%s')
x_test_250 = np.loadtxt('x_test_250.csv', delimiter=',').reshape((-1, 128, 9))
y_test_250 = np.loadtxt('y_test_250.csv', delimiter=',')

## Build model

In [3]:
model = Sequential()
model.add(Input(shape=(128, 9))) # 128 data points form sensors | 9 time steps
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on

# log NN details
wandb.init(
    project="Embedded-AI-Lab2-UCI-HAR",
    entity="juzay_and_co",
    name="Conv1D 64 | Conv1D 64 | Dropout | Flatten | Dense 100 | MaxPooling | Run 1",
    config=model.get_config()
)


# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(lr=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-19 09:30:58.397391: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-19 09:30:58.397566: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
wandb: Currently logged in as: citrovin (juzay_and_co). Use `wandb login --relogin` to force relogin


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 126, 64)           1792      
                                                                 
 conv1d_1 (Conv1D)           (None, 124, 64)           12352     
                                                                 
 dropout (Dropout)           (None, 124, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 62, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3968)              0         
                                                                 
 dense (Dense)               (None, 100)               396900    
                                                        

/opt/homebrew/Caskroom/miniforge/base/envs/embeddedAI/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Train model

In [4]:
model.fit(
    x_train, 
    y_train, 
    epochs=30, 
    validation_data=(x_test_250, y_test_250),
    batch_size=20,
    callbacks=[
        WandbMetricsLogger(log_freq=5),
        WandbCallback()
        ]
    )

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30


2023-01-19 09:31:04.857149: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/368 [..............................] - ETA: 2:52 - loss: 1.8623 - categorical_accuracy: 0.2000

2023-01-19 09:31:05.158171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

INFO:tensorflow:Assets written to: /Users/dalimwahby/Documents/EIT/UCA/Embedded AI/Labs/Lab2/UCI_HAR/wandb/run-20230119_093100-s4gnecaf/files/model-best/assets
wandb: Adding directory to artifact (/Users/dalimwahby/Documents/EIT/UCA/Embedded AI/Labs/Lab2/UCI_HAR/wandb/run-20230119_093100-s4gnecaf/files/model-best)... Done. 0.0s


368/368 [==============================] - 5s 13ms/step - loss: 0.1696 - categorical_accuracy: 0.9434 - val_loss: 0.1847 - val_categorical_accuracy: 0.9160
Epoch 9/30
368/368 [==============================] - 4s 11ms/step - loss: 0.1254 - categorical_accuracy: 0.9542 - val_loss: 0.1849 - val_categorical_accuracy: 0.9080
Epoch 10/30
368/368 [==============================] - 4s 11ms/step - loss: 0.1391 - categorical_accuracy: 0.9546 - val_loss: 0.3266 - val_categorical_accuracy: 0.8920
Epoch 11/30
366/368 [============================>.] - ETA: 0s - loss: 0.2648 - categorical_accuracy: 0.9320

INFO:tensorflow:Assets written to: /Users/dalimwahby/Documents/EIT/UCA/Embedded AI/Labs/Lab2/UCI_HAR/wandb/run-20230119_093100-s4gnecaf/files/model-best/assets


INFO:tensorflow:Assets written to: /Users/dalimwahby/Documents/EIT/UCA/Embedded AI/Labs/Lab2/UCI_HAR/wandb/run-20230119_093100-s4gnecaf/files/model-best/assets
wandb: Adding directory to artifact (/Users/dalimwahby/Documents/EIT/UCA/Embedded AI/Labs/Lab2/UCI_HAR/wandb/run-20230119_093100-s4gnecaf/files/model-best)... Done. 0.0s


368/368 [==============================] - 5s 13ms/step - loss: 0.2637 - categorical_accuracy: 0.9323 - val_loss: 0.1580 - val_categorical_accuracy: 0.9240
Epoch 12/30
368/368 [==============================] - 4s 11ms/step - loss: 0.1878 - categorical_accuracy: 0.9449 - val_loss: 0.2629 - val_categorical_accuracy: 0.8800
Epoch 13/30
368/368 [==============================] - 4s 11ms/step - loss: 0.2037 - categorical_accuracy: 0.9248 - val_loss: 0.2682 - val_categorical_accuracy: 0.8920
Epoch 14/30
368/368 [==============================] - 4s 11ms/step - loss: 0.3333 - categorical_accuracy: 0.9140 - val_loss: 0.1769 - val_categorical_accuracy: 0.8960
Epoch 15/30
368/368 [==============================] - 4s 10ms/step - loss: 0.2175 - categorical_accuracy: 0.9411 - val_loss: 0.2587 - val_categorical_accuracy: 0.8920
Epoch 16/30
368/368 [==============================] - 4s 11ms/step - loss: 0.2792 - categorical_accuracy: 0.9400 - val_loss: 0.1683 - val_categorical_accuracy: 0.9280
Epoc

## Evaluate model on small dataset

In [5]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

8/8 - 0s - loss: 0.2249 - categorical_accuracy: 0.8840 - 102ms/epoch - 13ms/step
65/88 [=====================>........] - ETA: 0s

2023-01-19 09:33:09.747716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


88/88 [==============================] - 0s 3ms/step
tf.Tensor(
[[427   5   6  28   0   0]
 [ 28 387  17  14   0   0]
 [  3   6 369  18   0   0]
 [  0  25   0 385  58   0]
 [  0   1   0 115 388   0]
 [  0   0   0   0   0 513]], shape=(6, 6), dtype=int32)


## Evaluate model on complete dataset

In [6]:
print(x_test.shape)
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

(2793, 128, 9)
88/88 - 1s - loss: 2.7638 - categorical_accuracy: 0.8840 - 589ms/epoch - 7ms/step
88/88 [==============================] - 0s 2ms/step
tf.Tensor(
[[427   5   6  28   0   0]
 [ 28 387  17  14   0   0]
 [  3   6 369  18   0   0]
 [  0  25   0 385  58   0]
 [  0   1   0 115 388   0]
 [  0   0   0   0   0 513]], shape=(6, 6), dtype=int32)


## Save trained model

In [7]:
model.save('uci-har_si4.h5')

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (Co